In [1]:
import tf.records as tfr
import ocr_dataset as ods
import moveread_ocr as mo
from haskellian import iter as I
import pure_cv as vc
import tensorflow as tf

2024-06-10 20:17:04.725520: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 20:17:04.925505: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 20:17:06.245570: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 20:17:09.027034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
names = ('test', 'train', 'validation')

def read_samples(name):
  path = f'data/written_name_{name}_v2.csv'
  with open(path) as f:
    for line in I.skip(1, f):
      file, lab = line.strip().split(',')
      yield f'data/{name}_v2/{name}/{file}', lab

samples = I.flatmap(read_samples, names).sync()

In [ ]:
paths, _ = I.unzip(samples)

In [ ]:
def read_samples(samples):
  for path, lab in samples:
    with open(path, 'rb') as f:
      data = f.read()
    yield data, lab

for i, batch in I.batch(50000, samples).enumerate():
  print(f'\r{i}', end='', flush=True)
  ods.create_tar(f'data/ocr-dataset/{i}', list(read_samples(batch)), images_ext='jpg')

In [4]:
def serialize(tup):
  i, (img, lab) = tup
  if i % 100 == 0:
    print(f'\r{i}', end='', flush=True)
  tensor = mo.parse_img(vc.decode(img))
  return tfr.serialize(mo.records.SCHEMA, image=tensor, label=tf.constant(lab.strip()))

In [5]:
for i in range(4, 9):
  print(i)
  ds = ods.Dataset.read(f'data/ocr-dataset/{i}')
  tfr.write(ds.samples().enumerate().map(serialize), f'data/tfrecords/{i}.tfrecord.gz', compression='GZIP')
  print()

4
0

2024-06-10 20:18:43.805084: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 20:18:44.044786: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


49900
5
49900
6
49900
7
49900
8
13700


In [ ]:
meta = tfr.Meta(files='*.tfrecord.gz', compression='GZIP', schema_=mo.records.SCHEMA, num_samples=ods.len(ods.glob('data/ocr-dataset/*')))

In [ ]:
with open('data/tfrecords/meta.json', 'w') as f:
  f.write(meta.model_dump_json(indent=2))